### Silver_layer_script

## Data Acces ussing App

# Data Loding


# **reading data**

In [0]:
display(dbutils.fs.ls("/mnt/"))

In [0]:
%fs ls dbfs:/mnt/bronze-layer

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_calender= spark.read.format('csv').option('header', True).option('inferSchema', True).load("/mnt/bronze-layer/AdventureWorks_Calendar/AdventureWorks_Calendar.csv")

df_calender.show()

In [0]:
#AdventureWorks_Customers/AdventureWorks_Customers.csv"
df_customer = spark.read.format('csv').option('header', True).option('inferSchema', True).load("/mnt/bronze-layer/AdventureWorks_Customers/AdventureWorks_Customers.csv")

#df_customer.show()
#df_customer.display()
display(df_customer)

In [0]:
#AdventureWorks_Product_Categories/AdventureWorks_Product_Categories.csv"
df_Product_Categories = spark.read.format('csv').option('header', True).option('inferSchema', True).load("/mnt/bronze-layer/AdventureWorks_Product_Categories/AdventureWorks_Product_Categories.csv")

df_Product_Categories.show()

In [0]:
# AdventureWorks_Products/AdventureWorks_Products.csv

df_AdventureWorks_Products = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Products/AdventureWorks_Products.csv")

df_AdventureWorks_Products.show()


In [0]:
#AdventureWorks_Returns/AdventureWorks_Returns.csv"

df_AdventureWorks_Returns = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Returns/AdventureWorks_Returns.csv")

df_AdventureWorks_Returns.show()   

In [0]:
#AdventureWorks_Sales_2015/AdventureWorks_Sales_2015.csv

df_AdventureWorks_Sales_2015 = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Sales_2015/AdventureWorks_Sales_2015.csv")
df_AdventureWorks_Sales_2015.show()              

In [0]:
#AdventureWorks_Sales_2016/AdventureWorks_Sales_2016.cvsv
df_AdventureWorks_Sales_2016 = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Sales_2016/AdventureWorks_Sales_2016.cvsv")
df_AdventureWorks_Sales_2016.show()


In [0]:
#AdventureWorks_Sales_2017/AdventureWorks_Sales_2017.csv

df_AdventureWorks_Sales_2017 = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Sales_2017/AdventureWorks_Sales_2017.csv")

df_AdventureWorks_Sales_2017.show()

In [0]:
#AdventureWorks_Territories/AdventureWorks_Territories.csv

df_AdventureWorks_Territories = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/AdventureWorks_Territories/AdventureWorks_Territories.csv")

df_AdventureWorks_Territories.show()

In [0]:
#Product_Subcategories/AdventureWorks_Product_Subcategories.csv
df_Product_Subcategories = spark.read.format('csv').option('header', True).option('inferSchema', True).load("dbfs:/mnt/bronze-layer/Product_Subcategories/AdventureWorks_Product_Subcategories.csv")

df_Product_Subcategories.show()

### Transformations


In [0]:
df_calender.display()

In [0]:
# create month column, day column

df_cal= df_calender.withColumn('Month', month(col('Date')))\
    .withColumn('Year', year(col('Date')))
df_cal.show()

### Calender


In [0]:
#save data into silver mode
df_cal.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_cal').save()

### Customer dataframe

In [0]:
df_customer.show()

In [0]:
#Concatenate MR, First name, last name # concat(lit(' '))

df_fullName = df_customer.withColumn('FullName', concat(lit(' '), col('Prefix'), lit(' '), col('FirstName'), lit(' '), col('LastName')))

df_fullName.show()

In [0]:
#Concatenate MR, First name, last name # concat_ws((' '), col())
df_fullName_concat_ws = df_customer.withColumn('FullName', concat_ws(' ', col('Prefix'), col('FirstName'), col('LastName')))
df_fullName_concat_ws.display()

# Save Customer data to the silver layer
df_fullName_concat_ws.write.format('parquet').mode('append').option('path', 'dbfs:/mnt/silver-layer/AdventureWorks_customer' ).save()

### Subcategories

In [0]:
df_Product_Subcategories.display()

df_Product_Subcategories.write.format('parquet').mode('append').option('path', 'dbfs:/mnt/silver-layer/AdventureWorks_Product_Subcategories').save()

**Product_data**

In [0]:
#df_AdventureWorks_Products.display()

df_product_SKU = df_AdventureWorks_Products.withColumn('ProductSKU', split(col('ProductSKU'),'-')[0])\
    .withColumn('ProductName', split(col('ProductName'), ' ')[0])

#df_product_SKU.show()

#write date into Silver layer
df_product_SKU.write.format('parquet').mode('append').option('path', 'dbfs:/mnt/silver-layer/AdventureWorks_Product').save()

### **Returns**

In [0]:
df_AdventureWorks_Returns.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_return').save()

In [0]:
df_AdventureWorks_Territories.display()

df_AdventureWorks_Territories.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Territories').save()

### SALE 2015

In [0]:
df_AdventureWorks_Sales_2015.display()

In [0]:
# Convert date to timestamp

df_Sale_2015 = df_AdventureWorks_Sales_2015.withColumn('StockDate', to_timestamp(col('StockDate')))

df_Sale_2015.display()
df_Sale_2015.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Sales').save()

In [0]:
#Replace S with T

df_Sale_2015_replace = df_Sale_2015.withColumn('OrderNumber', regexp_replace(col('OrderNumber'), 'S', 'T'))
df_Sale_2015_replace.show()

df_Sale_2015_replace.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Sales').save() 


In [0]:
# use of multiplicatuon functions

df_Sale_2015_replace = df_Sale_2015_replace.withColumn('Multiply', col('OrderLineItem')*(col('OrderQuantity')))
df_Sale_2015_replace.show()
df_Sale_2015_replace.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Sales').save() 

### SALE ANALYSIS

In [0]:
#Agrregation: How many order we recievd every day?

df_AdventureWorks_Sales_2015.groupBy('OrderDate').agg(count('OrderNumber').alias('Total_order')).display()
df_AdventureWorks_Sales_2015.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Sales_2015_visualization').save() 

Databricks visualization. Run in Databricks to view.

In [0]:
#want to see Product category by using pie chart

df_Product_Categories.display()
df_Product_Categories.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Product_Categories_visualization').save()

Databricks data profile. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
display(df_Product_Categories)
df_Product_Categories.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Product_Categories_visualization').save   


Databricks data profile. Run in Databricks to view.

In [0]:
df_AdventureWorks_Territories.display()

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
display(df_AdventureWorks_Territories)
df_AdventureWorks_Territories.write.format('parquet').mode('append').option('path','dbfs:/mnt/silver-layer/AdventureWorks_Territories_visualization').save()

Databricks data profile. Run in Databricks to view.